In [3]:
# Paruošia pūpsnių indeksus ML mokymui ir vertinimui 
# train_ind_lst ir validate_ind_lst ir įrašo juos į diską.
# Nebalansuotas variantas
# 
# Paruošti pūpsnių indeksų masyvai gali būti panaudoti tiesioginiam sekų 
# formavimui iš EKG įrašų (rec) skaičiavimo metu arba sekų Numpy masyvų formavimui

# Naudojami iš aplanko EKG įrašai ir failas all_beats_attr.csv: 
# {userNr, recordingNr, userId, recordingId, sample, symbol, labels},
# 
#  
# //////////////////////////////////////////////////////////////////////////////////////////////////////
#
import pandas as pd
import numpy as np
from numpy import loadtxt, savetxt
import matplotlib.pyplot as plt
from pathlib import Path
from icecream import ic
import json, time, sys

from mit2zive_util import create_dir
from mit2zive_util import anotaciju_pasiskirstymas_v2, runtime


my_os=sys.platform

print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'C:\DI\Data\MIT&ZIVE'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI/Data/MIT&ZIVE'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

#  MIT2ZIVE duomenų aplankas
db_folder = 'ZIVE&MIT_TST'

# Aplankas su MIT2ZIVE EKG įrašais (.npy) ir anotacijomis (.json)
rec_folder = 'records'

# Duomenų schemos aplanko pavadinimas
SCHEMA_FOLDER = 'schema_NSV_3_nbal'

# Kas kiek išvedamas apdorotų duomenų skaičius
show_period = 100

# Anotacijoms priskirtos klasės
selected_beats = {'N':0, 'S':1, 'V':2}

# Sekos kanalų skaičius
n_channels = 1

# Train imties pacientai
DS1 = [ 209, 228]

# Train imties pacientai
# DS1 = [101, 106, 108, 109, 112, 114, 115, 116, 118, 119, 122, 124, 201, 203, 205, 208, 209, 215, 220, 223,230]

# Validate imties pacientai
DS2 = [124]

# Validate imties pacientai
# DS2 = [100, 103, 105, 111, 113, 117, 121, 123, 200, 202, 210, 212, 213, 214, 219, 221, 222, 228, 231, 232, 233, 234]

# Kiekis pūpsnių, kurie atmetami EKG įrašo pradžioje ir pabaigoje
beats_skiped = 10
# ///////////////////////////////////////////////////////////////////////////////

# Anotacijų sąrašas
annot_list = [key for key in selected_beats]

# swapped_selected_beats
swapped_selected_beats = {v: k for k, v in selected_beats.items()}

# Nuoroda į MIT2ZIVE duomenų aplanką
db_path = Path(Duomenu_aplankas, db_folder)

# Nuoroda į aplanką su MIT2ZIVE EKG įrašais (.npa) ir anotacijomis (.json)
rec_dir = Path(db_path, rec_folder)

# Nuoroda į klasių schemos aplanką
SCHEMA_PATH = Path(db_path, SCHEMA_FOLDER)

# Nuoroda į modelio aplanką
# model_dir = Path(SCHEMA_PATH,'saved_model', model_folder)

# Nuskaitome failą info_create.json ir duomenų rinkinio parametrus
file_path = Path(rec_dir,'info_create.json')
with open(file_path) as json_file:
    info_create = json.load(json_file)

fs = info_create['fs'] # diskretizavimo dažnumas
subj_list =  info_create['subj_list'] # pacientų įrašų sąrašas


# Klasių skaičius
n_classes = len(selected_beats)

print("\nParuošia EKG pūpsnių indeksus ML mokymui ir vertinimui")
print("Nebalansuotas variantas")

print("\nBendras Zive duomenų aplankas: ", Duomenu_aplankas)
print("Zive EKG įrašų aplankas: ", db_folder)

print("Diskretizavimo dažnis: ", fs)
print("\nPacientų įrašų sąrašas:\n",subj_list)
print("Pacientų sąrašas mokymui:", DS1)
print("Pacientų sąrašas vertinimui:", DS2)

print("\nDuomenų schemos aplankas: ", SCHEMA_FOLDER)
print('Klasifikavimo schema:', selected_beats)
print('Klasių skaičius:', len(selected_beats))

print("\nParametrai pūpsnių kiekiui apriboti")
print(f"beats_skiped: {beats_skiped}")
print(f"Kiekviename EKG įraše praleisti {beats_skiped} pirmi ir {beats_skiped} paskutiniai pūpsniai")


OS in my system :  win32

Paruošia EKG pūpsnių indeksus ML mokymui ir vertinimui
Nebalansuotas variantas

Bendras Zive duomenų aplankas:  C:\DI\Data\MIT&ZIVE
Zive EKG įrašų aplankas:  ZIVE&MIT_TST
Diskretizavimo dažnis:  200

Pacientų įrašų sąrašas:
 ['124', '209', '228']
Pacientų sąrašas mokymui: [209, 228]
Pacientų sąrašas vertinimui: [124]

Duomenų schemos aplankas:  schema_NSV_3_nbal
Klasifikavimo schema: {'N': 0, 'S': 1, 'V': 2}
Klasių skaičius: 3

Parametrai pūpsnių kiekiui apriboti
beats_skiped: 10
Kiekviename EKG įraše atmetami' 10 'pirmi ir 10 paskutiniai pūpsniai


In [5]:
# from mit2zive_util import create_set_from_rec
from sklearn.preprocessing import StandardScaler
from numpy import asarray

start_time = time.time()

# Nuskaitome pūpsnių atributų freimą iš rec_dir aplanko
file_path = Path(rec_dir, 'all_beats_attr.csv')
all_beats_attr = pd.read_csv(file_path, index_col=0)

# Visų sekų indeksų sąrašas
all_beat_indices = all_beats_attr.index

DS = DS1 + DS2
all_ind_lst = []
for userNr in DS:
    selected_ind = all_beat_indices[all_beats_attr['userNr']==userNr]
    all_ind_lst.extend(selected_ind.to_list())

print("\nAnotacijų pasiskirstymas visuose duomenyse\n")
labels_table, labels_sums = anotaciju_pasiskirstymas_v2(all_beats_attr, ind_lst = all_ind_lst, cols_pattern=annot_list)
print(labels_table)
print("\nTotal:", labels_sums['All'])

# Suformuojame sekų masyvus klasifikatoriaus mokymui ir vertinimui

# MOKYMO IMTIS

# Suformuojame mokymo imties sekų indeksų sąrašą. Formuodami sąrašą
# kiekvienam pacientui eliminuojame pirmus ir paskutinius beats_skiped indeksus
train_ind_lst = []
for userNr in DS1:
    selected_ind = all_beat_indices[all_beats_attr['userNr']==userNr]
    selected_ind = selected_ind[beats_skiped:-beats_skiped]
    train_ind_lst.extend(selected_ind.to_list())

# Anotacijų pasiskirstymas mokymo duomenyse
print("\nAnotacijų pasiskirstymas mokymo duomenyse\n")
labels_table, labels_sums = anotaciju_pasiskirstymas_v2(all_beats_attr, ind_lst=train_ind_lst, cols_pattern=['N', 'S', 'V'])
print(labels_table)
print("\nTotal:", labels_sums['All'])

# Įrašome mokymo imties sekų indeksų sąrašą į diską
file_path = Path(SCHEMA_PATH, 'train_ind_lst.csv')
np.savetxt(file_path, np.array(train_ind_lst), delimiter=',', fmt='%d')
print("\nMokymo imties sekų indeksų sąrašas įrašytas į: ", file_path )

# Suformuojame mokymo sekų ir jų klasių numerių masyvus
# X_train, y_train = create_set_from_rec(rec_dir, all_beats_attr, train_ind_lst, window_left_side, window_right_side)
# print("\nMokymo imties dimensijos")
# print(f"{X_train.shape=} {y_train.shape=}")

# Įrašome mokymui skirtus masyvus į npz file diske
# file_path = Path(SCHEMA_PATH, 'train_set.npz')
# np.savez_compressed(file_path, X=X_train, y=y_train)
# print("\nMokymo sekų ir jų klasių numerių masyvas įrašytas: į ", file_path )


# VERTINIMO IMTIS
# Suformuojame vertinimo imties sekų indeksų sąrašą. Formuodami sąrašą
# kiekvienam pacientui eliminuojame pirmus ir paskutinius beats_skiped indeksus
validate_ind_lst = []
for userNr in DS2:
    selected_ind = all_beat_indices[all_beats_attr['userNr']==userNr]
    selected_ind = selected_ind[beats_skiped:-beats_skiped]
    validate_ind_lst.extend(selected_ind.to_list())

# Anotacijų pasiskirstymas vertinimo duomenyse
print("\nAnotacijų pasiskirstymas vertinimo duomenyse\n")
labels_table, labels_sums = anotaciju_pasiskirstymas_v2(all_beats_attr, ind_lst=validate_ind_lst, cols_pattern=['N', 'S', 'V'])
print(labels_table)
print("\nTotal:", labels_sums['All'])

# Įrašome vertinimo imties sekų indeksų sąrašą į diską
file_path = Path(SCHEMA_PATH, 'validate_ind_lst.csv')
np.savetxt(file_path, np.array(validate_ind_lst), delimiter=',', fmt='%d')
print("\nVertinimo imties sekų indeksų sąrašas įrašytas į: ", file_path )

# Suformuojame mokymo sekų ir jų klasių numerių masyvus
# X_validate, y_validate = create_set_from_rec(rec_dir, all_beats_attr, validate_ind_lst, window_left_side, window_right_side)
# print("\nVertinimo imties dimensijos")
# print(f"{X_validate.shape=} {y_validate.shape=}")

# Įrašome vertinimui skirtus masyvus į npz file diske
# file_path = Path(SCHEMA_PATH, 'validate_set.npz')
# np.savez_compressed(file_path, X=X_validate, y=y_validate)
# print("\nVertinimo sekų ir jų klasių numerių masyvas įrašytas: į ", file_path )

end_time = time.time()
print('\n')
runtime(end_time-start_time)

# Suformuojame info_prep masyvą ir įrašome į diską

info = {
    'fs': fs,
    'subj_list':subj_list,
    'selected_beats':selected_beats,
    'beats_skiped':beats_skiped
    }

file_path = Path(SCHEMA_PATH,'info_prep.json')
with open(file_path, 'w') as f:
    json.dump(info, f)
    
print("\nSchemos parametrai info.json yra aplanke: ", file_path, "\n" )

print("\nPabaiga.............")



Anotacijų pasiskirstymas visuose duomenyse

symbol     N    S    V
userNr                
124     1536   31   47
209     2621  383    1
228     1688    3  362
All     5845  417  410

Total: 6672

Anotacijų pasiskirstymas mokymo duomenyse

symbol     N    S    V
userNr                
209     2601  383    1
228     1671    3  359
All     4272  386  360

Total: 5018

Mokymo imties sekų indeksų sąrašas įrašytas į:  C:\DI\Data\MIT&ZIVE\ZIVE&MIT_TST\schema_NSV_3_nbal\train_ind_lst.csv

Anotacijų pasiskirstymas vertinimo duomenyse

symbol     N   S   V
userNr              
124     1516  31  47
All     1516  31  47

Total: 1594

Vertinimo imties sekų indeksų sąrašas įrašytas į:  C:\DI\Data\MIT&ZIVE\ZIVE&MIT_TST\schema_NSV_3_nbal\validate_ind_lst.csv


Runtime: 00:00:00

Schemos parametrai info.json yra aplanke:  C:\DI\Data\MIT&ZIVE\ZIVE&MIT_TST\schema_NSV_3_nbal\info_prep.json 


Pabaiga.............
